In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.display import clear_output

In [2]:
def scrap_indeed_offer(URL,city_set,nb_resu_par_ville=1,extension="https://www.indeed.com"):
    columns = ["ville", "utituler_offre","description","entreprise","lacalisation", "resume"]
    sample_df = pd.DataFrame(columns = columns)
    for city in city_set:
      for start in range(0,nb_resu_par_ville, 10):
          page = requests.get(URL+"&l="+str(city)+'&start='+str(start))
          soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
          for div in soup.find_all(name="div", attrs={"class":"row"}): 
            #specification du nombre de lignes
            num = (len(sample_df) + 1) 
            
            job_post = [] 
            job_post.append(city) 
            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
              job_post.append(a["title"])
             # recuperer site sousjacent contenant le descriptif de l'offre 
              url_fils=extension+a["href"]
              page_fils=requests.get(url_fils)
              soup_fils=BeautifulSoup(page_fils.text,"lxml",from_encoding="utf-8")
              div_class="jobsearch-JobComponent-description icl-u-xs-mt--md"
              for desc in soup_fils.find_all(name="div",attrs={"class":div_class}):
                  if len(desc.text)>0:
                     job_post.append(desc.text)
                  else:
                     job_post.append("")
              #NOm de la companie
              company = div.find_all(name="span", attrs={"class":"company"}) 
              if len(company) > 0: 
                  for b in company:
                    job_post.append(b.text.strip()) 
              else: 
                sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
                for span in sec_try:
                    job_post.append(span.text) 
        #localisation
              c = div.findAll("span", attrs={"class": "location"}) 
              if len(c)>0:
                for span in c: 
                    job_post.append(span.text)
              else:
                    job_post.append("pas localisé")
        #resumer
              d = div.findAll("span", attrs={"class": "summary"}) 
              for span in d:
                if len(span.text.strip())>0:
                    job_post.append(span.text.strip())
                else:
                    job_post.append("")        
        #mise a jours de notre dataframe
            clear_output(wait=True)
            print("nb_boocle:{}".format(num))
            print("len job_post={}".format(len(job_post)))
            print(job_post)
            if len(job_post)==sample_df.shape[1]:
                sample_df.loc[num,:] = job_post
    t=["ville", "utituler_offre","entreprise","localisation", "resume","description"]
    res=sample_df.loc[:,t]
    return(res)

In [3]:
nb_resu_par_ville = 40
us_city=pd.read_table("us_city.txt",sep=",")
ville_usa=[k.replace(" ","+") for k in us_city.city]
city_set =ville_usa.copy()
city_set=city_set
 
URL="https://www.indeed.com/jobs?q=network+engineer"
t1=time.time()
res_network=scrap_indeed_offer(URL,city_set,nb_resu_par_ville)
delta=time.time()-t1
print("Durée total réelle:{}".format(delta))

res_duplicate_network_usa=res_network.drop_duplicates()
print(res_duplicate_network_usa.shape)
res_duplicate_network_usa.to_csv("network_USA.csv",index=False)

nb_boocle:36661
len job_post=6
['Panama+City', 'SharePoint Consultant', 'JOB DESCRIPTIONProvide full time on-site advanced SharePoint consulting, development and support services for a Solution Foundry government customer in the Panama City, Fl area.MUST-HAVESThe ideal candidate possesses the capabilities and experience listed below and can demonstrate ability to expertly apply these skills in a SharePoint environment.3+ recent years of primary work experience as a Microsoft SharePoint power user and/or developer2+ years of experience supporting federal government customer(s)Ability to design, develop and implement advanced SharePoint 2013, 2016 or Office 365 solutionsAdvanced knowledge of all out-of-the-box SharePoint features, functionality and configurationsExperience managing SharePoint site content, sites/site collections, permissions and groups, and workflowsExperience customizing SharePoint sites using SharePoint DesignerAbility to analyze data and determine opportunities for Sh

/home/tantely/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


(27759, 6)
